In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.nn import to_hetero, GraphConv

In [9]:
#read in the credit card transaction data
# The data is from IBM's credit card fraud detection dataset on Kaggle
# https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions.

df = pd.read_csv('data/credit_card_transactions-ibm_v2.csv')

# Take a small sample of the data for computational efficiency
# To avoid extreme class imbalance, I will force 50% of the sample to be fraud

s1 = df[df['Is Fraud?'] == 'No'].sample(n=5000, random_state=4, replace=False)
s2 = df[df['Is Fraud?'] == 'Yes'].sample(n=5000, random_state=4, replace=False)

s = pd.concat([s1, s2]).reset_index(drop=True)

#take a smaller sample to act as the test set

s1 = df[df['Is Fraud?'] == 'No'].sample(n=1000, random_state=5, replace=False)
s2 = df[df['Is Fraud?'] == 'Yes'].sample(n=1000, random_state=5, replace=False)

test = pd.concat([s1, s2]).reset_index(drop=True)

#add user information from user file to the data
udf = pd.read_csv('data/sd254_users.csv')
udf.reset_index(inplace=True)
udf = udf[['index','Current Age','Retirement Age','Gender','State','Zipcode','Yearly Income - Person','Total Debt','FICO Score','Num Credit Cards']]
s = pd.merge(s, udf, how='left', left_on='User',right_on='index')
test = pd.merge(test, udf, how='left', left_on='User',right_on='index')

# Drop zip and state as they are missing values
s.drop(columns=['Zip'], inplace=True)
test.drop(columns=['Zip'], inplace=True)

# Fill in value where no error
s['Errors?'].fillna('N', inplace=True)
test['Errors?'].fillna('N', inplace=True)

# Convert Is Fraud to binary
s['Is Fraud?'] = s['Is Fraud?'].map({'Yes': 1, 'No': 0})
test['Is Fraud?'] = test['Is Fraud?'].map({'Yes': 1, 'No': 0})

# Separate time into hours and minutes
s['Hour'] = s['Time'].str.split(':').str[0].astype(int)
s['Minute'] = s['Time'].str.split(':').str[1].astype(int)
test['Hour'] = test['Time'].str.split(':').str[0].astype(int)
test['Minute'] = test['Time'].str.split(':').str[1].astype(int)

# Drop time
s.drop(columns='Time', inplace=True)
test.drop(columns='Time', inplace=True)

#convert dollars to floats
for col in ['Yearly Income - Person', 'Total Debt', 'Amount']:
    s[col] = s[col].str.replace('$', '').astype(float)
    test[col] = test[col].str.replace('$', '').astype(float)

# replace unknown merhcant states with unknown
s['Merchant State']  = s['Merchant State'] .fillna('unknown')
test['Merchant State']  = test['Merchant State'] .fillna('unknown')
# create an indicator for unknown state
s['Unknown State'] = s.apply(lambda row: 1 if row['Merchant State'] == 'unknown' else 0, axis=1)
test['Unknown State'] = test.apply(lambda row: 1 if row['Merchant State'] == 'unknown' else 0, axis=1)

# Convert categorical to numeric with label encoder 
# (for simplicity using label encoding despite introduction of ordinality)
le = LabelEncoder()
s['Use Chip'] = le.fit_transform(s['Use Chip'])
test['Use Chip'] = le.fit_transform(test['Use Chip'])
s['Merchant Name'] = le.fit_transform(s['Merchant Name'])
test['Merchant Name'] = le.fit_transform(test['Merchant Name'])
s['Merchant City'] = le.fit_transform(s['Merchant City'])
test['Merchant City'] = le.fit_transform(test['Merchant City'])
s['Errors?'] = le.fit_transform(s['Errors?'])
test['Errors?'] = le.fit_transform(test['Errors?'])
s['User'] = le.fit_transform(s['User'])
test['User'] = le.fit_transform(test['User'])
s['Gender'] = le.fit_transform(s['Gender'])
test['Gender'] = le.fit_transform(test['Gender'])
s['State'] = le.fit_transform(s['State'])
test['State'] = le.fit_transform(test['State'])
s['Zipcode'] = le.fit_transform(s['Zipcode'])
test['Zipcode'] = le.fit_transform(test['Zipcode'])
# create unique id for transaction from 0 to n-1
s['Transaction ID'] = range(0, len(s))
test['Transaction ID'] = range(0, len(test))
s.info()

C:\Users\wren1\AppData\Local\Temp\ipykernel_25648\3780533932.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  s['Errors?'].fillna('N', inplace=True)
C:\Users\wren1\AppData\Local\Temp\ipykernel_25648\3780533932.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User                    10000 non-null  int64  
 1   Card                    10000 non-null  int64  
 2   Year                    10000 non-null  int64  
 3   Month                   10000 non-null  int64  
 4   Day                     10000 non-null  int64  
 5   Amount                  10000 non-null  float64
 6   Use Chip                10000 non-null  int32  
 7   Merchant Name           10000 non-null  int64  
 8   Merchant City           10000 non-null  int32  
 9   Merchant State          10000 non-null  object 
 10  MCC                     10000 non-null  int64  
 11  Errors?                 10000 non-null  int32  
 12  Is Fraud?               10000 non-null  int64  
 13  index                   10000 non-null  int64  
 14  Current Age             10000 non-null 

In [10]:
# 1. Logistic Regression for Predicting Fraud (baseline)

# We will run a logistic regression to predict fraud as a baseline for comparison to the GNN model

X = s.drop(columns=['Is Fraud?', 'User', 'Merchant Name','Transaction ID','Merchant State'])
y = s['Is Fraud?']
#scale the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

lr = LogisticRegression(max_iter=1000)
lr.fit(X,y)
y_pred = lr.predict(X)

print('Logistic Regression Training Results')
print('accuracy',accuracy_score(y, y_pred))
print('precision',precision_score(y, y_pred))
print('recall',recall_score(y, y_pred))
print('f1 score',f1_score(y, y_pred))

X_test = test.drop(columns=['Is Fraud?', 'User', 'Merchant Name','Transaction ID','Merchant State'])
y_test = test['Is Fraud?']

X_test = scaler.transform(X_test)

y_pred = lr.predict(X_test)

print('Logistic Regression Test Results')
print('accuracy',accuracy_score(y_test, y_pred))
print('precision',precision_score(y_test, y_pred))
print('recall',recall_score(y_test, y_pred))
print('f1 score',f1_score(y_test, y_pred))


Logistic Regression Training Results
accuracy 0.764
precision 0.8139866793529972
recall 0.6844
f1 score 0.7435897435897436
Logistic Regression Test Results
accuracy 0.769
precision 0.8345771144278606
recall 0.671
f1 score 0.7439024390243902


In [11]:
# 2. GNN for Predicting Fraud

# create a heterogenous graph
data = HeteroData()
test_data = HeteroData()
# add nodes
data['user'].num_nodes = s['User'].nunique()
data['merchant'].num_nodes = s['Merchant Name'].nunique()
data['transaction'].num_nodes = s['Transaction ID'].nunique()

test_data['user'].num_nodes = test['User'].nunique()
test_data['merchant'].num_nodes = test['Merchant Name'].nunique()
test_data['transaction'].num_nodes = test['Transaction ID'].nunique()

#create edges

data['user','makes','transaction'].edge_index = torch.tensor(s[['User', 'Transaction ID']].values.T, dtype=torch.long)
data['transaction','reaches','merchant'].edge_index = torch.tensor(s[['Transaction ID', 'Merchant Name']].values.T, dtype=torch.long)

test_data['user','makes','transaction'].edge_index = torch.tensor(test[['User', 'Transaction ID']].values.T, dtype=torch.long)
test_data['transaction','reaches','merchant'].edge_index = torch.tensor(test[['Transaction ID', 'Merchant Name']].values.T, dtype=torch.long)

# add transaction features
transaction_features = torch.tensor(s.drop(columns=['Is Fraud?', 'User', 'Merchant Name','Transaction ID','Merchant State']).values, dtype=torch.float)
test_transaction_features = torch.tensor(test.drop(columns=['Is Fraud?', 'User', 'Merchant Name','Transaction ID','Merchant State']).values, dtype=torch.float) 

# print(transaction_features.shape)
data['transaction'].x = transaction_features
test_data['transaction'].x = test_transaction_features

target = torch.tensor(s['Is Fraud?'].values, dtype=torch.float).unsqueeze(0).T
test_target = torch.tensor(test['Is Fraud?'].values, dtype=torch.float).unsqueeze(0).T
# print(target.shape)
data['transaction'].y = target
test_data['transaction'].y = test_target

#add user node features
user_feats = s[['Current Age','Retirement Age','Gender','State','Zipcode','Yearly Income - Person','Total Debt','FICO Score','Num Credit Cards']].drop_duplicates().reset_index().drop(columns='index',axis=1)
user_features = torch.tensor(user_feats.values,dtype=torch.float)

test_user_feats = test[['Current Age','Retirement Age','Gender','State','Zipcode','Yearly Income - Person','Total Debt','FICO Score','Num Credit Cards']].drop_duplicates().reset_index().drop(columns='index',axis=1)
test_user_features = torch.tensor(test_user_feats.values,dtype=torch.float)

# print(user_features.shape)
data['user'].x = user_features
test_data['user'].x = test_user_features

#add merchant node features (just one for unknown states)
x = s[['Merchant Name','Unknown State']].drop_duplicates()
x_test = test[['Merchant Name','Unknown State']].drop_duplicates()
y = x.groupby('Merchant Name').prod()
y_test = x_test.groupby('Merchant Name').prod()

#merchant data is not 1x1 to merchant id, so created a column to represent if the merchant has no known states or if all states are known
data['merchant'].x= torch.tensor(y['Unknown State'],dtype=torch.float).unsqueeze(0).T
test_data['merchant'].x= torch.tensor(y_test['Unknown State'],dtype=torch.float).unsqueeze(0).T

# Add reverse edges and normalize features
data = T.ToUndirected()(data)
data = T.NormalizeFeatures()(data)
test_data = T.ToUndirected()(test_data)
test_data = T.NormalizeFeatures()(test_data)

node_types, edge_types = data.metadata()


print(node_types)

print(edge_types)

print(data)

['user', 'merchant', 'transaction']
[('user', 'makes', 'transaction'), ('transaction', 'reaches', 'merchant'), ('transaction', 'rev_makes', 'user'), ('merchant', 'rev_reaches', 'transaction')]
HeteroData(
  user={
    num_nodes=1511,
    x=[1511, 9],
  },
  merchant={
    num_nodes=2010,
    x=[2010, 1],
  },
  transaction={
    num_nodes=10000,
    x=[10000, 22],
    y=[10000, 1],
  },
  (user, makes, transaction)={ edge_index=[2, 10000] },
  (transaction, reaches, merchant)={ edge_index=[2, 10000] },
  (transaction, rev_makes, user)={ edge_index=[2, 10000] },
  (merchant, rev_reaches, transaction)={ edge_index=[2, 10000] }
)


In [12]:
# Define the GNN model with two layers of GraphConv layers
class GNN(torch.nn.Module):

    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        # -1 is lazy initialization of the shape, which will be inferred from the first forward pas
        self.conv1 = GraphConv((-1, -1), hidden_channels)
        self.conv2 = GraphConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

model = GNN(hidden_channels=64, out_channels=2)

# Convert the model to a heterogenous model
# The model will now take a dictionary of node features and a dictionary of edge indices
model = to_hetero(model, data.metadata(), aggr='sum')

 
# Define the optimizer to be used in training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

 
# Create dictionaries of node features and edge indices from graph data
node_x_dict = {'user':data['user'].x, 'transaction': data['transaction'].x, 'merchant': data['merchant'].x}

edge_ind_dict =  {('user', 'makes', 'transaction'): data['user', 'makes', 'transaction'].edge_index,
                  ('transaction', 'reaches', 'merchant'): data['transaction', 'reaches', 'merchant'].edge_index,
                  ('transaction', 'rev_makes', 'user'): data['transaction', 'rev_makes', 'user'].edge_index,
                  ('merchant', 'rev_reaches', 'transaction'): data['merchant', 'rev_reaches', 'transaction'].edge_index
                 }

 
# Train the model
model.train()
for epoch in range(2000):
    # Zero the gradients
    optimizer.zero_grad()
    # Perform a forward pass
    out = model(node_x_dict, edge_ind_dict)
    # Compute the loss based on the nodes in the 'transaction' type
    actual = data['transaction'].y.squeeze().long()
    loss = F.cross_entropy(out['transaction'], actual)
    # Perform a backward pass
    loss.backward()
    # Update the parameters
    optimizer.step()
    # Print the loss at every 100th epoch
    # This is useful for tracking convergence
    if epoch % 100 == 0:
        print(float(loss))

 
# Evaluate the model on the training data
model.eval()
y_pred = model(node_x_dict, edge_ind_dict)
y_pred = y_pred['transaction'].argmax(dim=1)

y = data['transaction'].y.squeeze().long()

print('GNN Training Results')
print('accuracy',accuracy_score(y, y_pred))
print('precision',precision_score(y, y_pred))
print('recall',recall_score(y, y_pred))
print('f1 score',f1_score(y, y_pred))

# Evaluate the model on the test data

node_x_dict = {'user':test_data['user'].x, 'transaction': test_data['transaction'].x, 'merchant': test_data['merchant'].x}  
edge_ind_dict =  {('user', 'makes', 'transaction'): test_data['user', 'makes', 'transaction'].edge_index,
                    ('transaction', 'reaches', 'merchant'): test_data['transaction', 'reaches', 'merchant'].edge_index,
                    ('transaction', 'rev_makes', 'user'): test_data['transaction', 'rev_makes', 'user'].edge_index,
                    ('merchant', 'rev_reaches', 'transaction'): test_data['merchant', 'rev_reaches', 'transaction'].edge_index
                     }

model.eval()
y_pred = model(node_x_dict, edge_ind_dict)
y_pred = y_pred['transaction'].argmax(dim=1)

y = test_data['transaction'].y.squeeze().long()

print('GNN Test Results')
print('accuracy',accuracy_score(y, y_pred))
print('precision',precision_score(y, y_pred))
print('recall',recall_score(y, y_pred))
print('f1 score',f1_score(y, y_pred))



1.9666961431503296
0.6065921783447266
0.5640533566474915
0.5651065707206726
0.5364245176315308
0.5523791313171387
0.5074954628944397
0.5121963024139404
0.5115290880203247
0.485074907541275
0.47643759846687317
0.46380966901779175
0.4548254907131195
0.44800257682800293
0.4446171820163727
0.44762134552001953
0.4357404410839081
0.43780389428138733
0.42293277382850647
0.4246046245098114
GNN Training Results
accuracy 0.8154
precision 0.7934499441756606
recall 0.8528
f1 score 0.8220551378446115
GNN Test Results
accuracy 0.5265
precision 0.5317365269461077
recall 0.444
f1 score 0.4839237057220708
